# Storm Detection and Tracking
## Code Written by: Eric Oliver
### Edited by Robert Fritzen for NIU WCS Storm Tracking Project

**Python Script Used for:** Fritzen, R., V. Lang, and V. Gensini, 2020: Trends and Variability of North American Extratropical Cyclones: 1979–2019. *J. Appl. Met. Cli.*, Submitted

The original script (https://github.com/ecjoliver/stormTracking) was written to detect cyclones and anticyclones for the twentieth century reanalysis (20CR) dataset, although it employed Python 2 and a few packages that have since lost support or feature now depricated methods.

This notebook features the edited scripts, along with Python 3 compliance and the introduction of replacement packages that are more "properly" maintained. This notebook can be run on a clean anaconda-3 installation with only a few extra packages required.

Required Packages:
* numpy
* scipy
* matplotlib
* netCDF4

This script has been tested (And used) on the North American Regional Reanalysis (NARR) as well as the ERA-5 datasets.

In [ ]:
import numpy as np
import scipy as sp
import scipy.ndimage as ndimage
from datetime import date
from itertools import repeat
from matplotlib import pyplot as plt
import matplotlib.path as mplPath
from netCDF4 import Dataset
from functools import partial
import multiprocessing
import os
import itertools
import time

## Required Utility Functions
Run this code block to add required functions to the notebook

In [ ]:
def interp_lat_lon(iReal, jReal, lon, lat):
    '''
    This interpolation routine functions by stripping the decimal value
    from the i/j values and then interpolating between the two nearest 
    points along the lat/lon
    '''
    iDec = iReal % 1
    jDec = jReal % 1
    iInt = int(np.floor(iReal))
    jInt = int(np.floor(jReal))

    lonLow = lon[iInt, jInt]
    lonHigh = lon[iInt + 1, jInt + 1]
    latLow = lat[iInt, jInt]
    latHigh = lat[iInt + 1, jInt + 1]

    difLon = np.abs(lonHigh - lonLow)
    difLat = np.abs(latHigh - latLow)

    distLon = difLon * iDec
    distLat = difLat * jDec

    trueLon = lonLow + distLon
    trueLat = latLow + distLat

    return trueLon, trueLat

def distance_matrix(lons,lats):
    '''
    Calculates the distances (in km) between any two cities based on the formulas
    c = sin(lati1)*sin(lati2)+cos(longi1-longi2)*cos(lati1)*cos(lati2)
    d = EARTH_RADIUS*Arccos(c)
    where EARTH_RADIUS is in km and the angles are in radians.
    Source: http://mathforum.org/library/drmath/view/54680.html
    This function returns the matrix.
    '''

    EARTH_RADIUS = 6378.1
    X = len(lons)
    Y = len(lats)
    assert X == Y, 'lons and lats must have same number of elements'

    d = np.zeros((X,X))

    #Populate the matrix.
    for i2 in range(len(lons)):
        lati2 = lats[i2]
        loni2 = lons[i2]
        c = np.sin(np.radians(lats)) * np.sin(np.radians(lati2)) + \
            np.cos(np.radians(lons-loni2)) * \
            np.cos(np.radians(lats)) * np.cos(np.radians(lati2))
        d[c<1,i2] = EARTH_RADIUS * np.arccos(c[c<1])

    return d

def detect_storms(field, lon, lat, res, Npix_min, cyc):
    '''
    Detect storms present in field which satisfy the criteria.
    Algorithm is an adaptation of an eddy detection algorithm,
    outlined in Chelton et al., Prog. ocean., 2011, App. B.2,
    with modifications needed for storm detection.

    field is a 2D array specified on grid defined by lat and lon.

    res is the horizontal grid resolution in degrees of field

    Npix_min is the minimum number of pixels within which an
    extremum of field must lie (recommended: 9).

    cyc = 'cyclonic' or 'anticyclonic' specifies type of system
    to be detected (cyclonic storm or high-pressure systems)

    Function outputs lon, lat coordinates of detected storms
    '''
    len_deg_lat = 111.325 # length of 1 degree of latitude [km]

    lon_storms = np.array([])
    lat_storms = np.array([])
    amp_storms = np.array([])

    # Strip out the missing value flag
    field[field==-9.969209968386869e+36] = np.nan

    # ssh_crits is an array of ssh levels over which to perform storm detection loop
    # ssh_crits increasing for 'cyclonic', decreasing for 'anticyclonic'
    ssh_crits = np.linspace(np.nanmin(field), np.nanmax(field), 200)
    ssh_crits.sort()
    if cyc == 'anticyclonic':
        ssh_crits = np.flipud(ssh_crits)

    # loop over ssh_crits and remove interior pixels of detected storms from subsequent loop steps
    for ssh_crit in ssh_crits:

        # 1. Find all regions with eta greater (less than) than ssh_crit for anticyclonic (cyclonic) storms (Chelton et al. 2011, App. B.2, criterion 1)
        if cyc == 'anticyclonic':
            regions, nregions = ndimage.label( (field>ssh_crit).astype(int) )
        elif cyc == 'cyclonic':
            regions, nregions = ndimage.label( (field<ssh_crit).astype(int) )

        for iregion in range(nregions): 
    # 2. Calculate number of pixels comprising detected region, reject if not within >= Npix_min
            region = (regions==iregion+1).astype(int)
            region_Npix = region.sum()
            storm_area_within_limits = (region_Npix >= Npix_min)

    # 3. Detect presence of local maximum (minimum) for anticylones (cyclones), reject if non-existent
            interior = ndimage.binary_erosion(region)
            exterior = region.astype(bool) ^ interior
            if interior.sum() == 0:
                continue
            if cyc == 'anticyclonic':
                has_internal_ext = field[interior].max() > field[exterior].max()
            elif cyc == 'cyclonic':
                has_internal_ext = field[interior].min() < field[exterior].min()

    # 4. Find amplitude of region, reject if < amp_thresh
            if cyc == 'anticyclonic':
                amp_abs = field[interior].max()
                amp = amp_abs - field[exterior].mean()
            elif cyc == 'cyclonic':
                amp_abs = field[interior].min()
                amp = field[exterior].mean() - amp_abs
            amp_thresh = np.abs(np.diff(ssh_crits)[0])
            is_tall_storm = amp >= amp_thresh

    # Quit loop if these are not satisfied
            if np.logical_not(storm_area_within_limits * has_internal_ext * is_tall_storm):
                continue

    # Detected storms:
            if storm_area_within_limits * has_internal_ext * is_tall_storm:
                # find centre of mass of storm
                storm_object_with_mass = field * region
                storm_object_with_mass[np.isnan(storm_object_with_mass)] = 0
                j_cen, i_cen = ndimage.center_of_mass(storm_object_with_mass)
                lon_cen, lat_cen = interp_lat_lon(j_cen, i_cen, lon, lat)
                # Save storm
                lon_storms = np.append(lon_storms, lon_cen)
                lat_storms = np.append(lat_storms, lat_cen)
                # assign (and calculated) amplitude, area, and scale of storms
                amp_storms = np.append(amp_storms, amp_abs)
                # remove its interior pixels from further storm detection
                storm_mask = np.ones(field.shape)
                storm_mask[interior.astype(int)==1] = np.nan
                field = field * storm_mask

    return lon_storms, lat_storms, amp_storms

def storms_list(lon_storms_a, lat_storms_a, amp_storms_a, lon_storms_c, lat_storms_c, amp_storms_c):
    '''
    Creates list detected storms
    '''

    storms = []

    for ed in range(len(lon_storms_c)):
        storm_tmp = {}
        storm_tmp['lon'] = np.append(lon_storms_a[ed], lon_storms_c[ed])
        storm_tmp['lat'] = np.append(lat_storms_a[ed], lat_storms_c[ed])
        storm_tmp['amp'] = np.append(amp_storms_a[ed], amp_storms_c[ed])
        storm_tmp['type'] = list(repeat('anticyclonic',len(lon_storms_a[ed]))) + list(repeat('cyclonic',len(lon_storms_c[ed])))
        storm_tmp['N'] = len(storm_tmp['lon'])
        storms.append(storm_tmp)

    return storms

def storms_init(det_storms, year, month, day, hour):
    '''
    Initializes list of storms. The ith element of output is
    a dictionary of the ith storm containing information about
    position and size as a function of time, as well as type.
    '''

    storms = []

    for ed in range(det_storms[0][0]['N']):
        storm_tmp = {}
        storm_tmp['lon'] = np.array([det_storms[0][0]['lon'][ed]])
        storm_tmp['lat'] = np.array([det_storms[0][0]['lat'][ed]])
        storm_tmp['amp'] = np.array([det_storms[0][0]['amp'][ed]])
        storm_tmp['type'] = det_storms[0][0]['type'][ed]
        storm_tmp['year'] = np.array([year[0]])
        storm_tmp['month'] = np.array([month[0]])
        storm_tmp['day'] = np.array([day[0]])
        storm_tmp['hour'] = np.array([hour[0]])
        storm_tmp['exist_at_start'] = True
        storm_tmp['terminated'] = False
        storms.append(storm_tmp)

    return storms

def len_deg_lon(lat):
    '''
    Returns the length of one degree of longitude (at latitude
    specified) in km.
    '''

    R = 6371. # Radius of Earth [km]

    return (np.pi/180.) * R * np.cos( lat * np.pi/180. )


def len_deg_lat():
    '''
    Returns the length of one degree of latitude in km.
    '''
    return 111.325 # length of 1 degree of latitude [km]


def latlon2km(lon1, lat1, lon2, lat2):
    '''
    Returns the distance, in km, between (lon1, lat1) and (lon2, lat2)
    '''

    EARTH_RADIUS = 6371. # Radius of Earth [km]
    c = np.sin(np.radians(lat1)) * np.sin(np.radians(lat2)) + np.cos(np.radians(lon1-lon2)) * np.cos(np.radians(lat1)) * np.cos(np.radians(lat2))
    d = EARTH_RADIUS * np.arccos(c)

    return d

# Robert Note: Manipulating the prop_speed argument will control the search radius
def track_storms(storms, det_storms, tt, year, month, day, hour, dt, prop_speed=140.):
    '''
    Given a set of detected storms as a function of time (det_storms)
    this function will update tracks of individual storms at time step
    tt in variable storms

    dt indicates the time step of the underlying data (in hours)

    prop_speed indicates the maximum storm propagation speed (in km/hour)
    '''

    # List of unassigned storms at time tt
    unassigned = list(range(det_storms[tt][0]['N']))

    # For each existing storm (t<tt) loop through unassigned storms and assign to existing storm if appropriate

    for ed in range(len(storms)):

        # Check if storm has already been terminated

        if not storms[ed]['terminated']:

            # Define search region around centroid of existing storm ed at last known position

            x0 = storms[ed]['lon'][-1] # [deg. lon]
            y0 = storms[ed]['lat'][-1] # [deg. lat]

            # Find all storm centroids in search region at time tt

            is_near = latlon2km(x0, y0, det_storms[tt][0]['lon'][unassigned], det_storms[tt][0]['lat'][unassigned]) <= prop_speed*dt

            # Check if storms' type is the same as original storm

            is_same_type = np.array([det_storms[tt][0]['type'][i] == storms[ed]['type'] for i in unassigned])

            # Possible storms are those which are near and of the same type

            possibles = is_near * is_same_type
            if possibles.sum() > 0:

                # Of all found storms, accept only the nearest one

                dist = latlon2km(x0, y0, det_storms[tt][0]['lon'][unassigned], det_storms[tt][0]['lat'][unassigned])
                nearest = dist == dist[possibles].min()
                next_storm = unassigned[np.where(nearest * possibles)[0][0]]

                # Add coordinatse and properties of accepted storm to trajectory of storm ed

                storms[ed]['lon'] = np.append(storms[ed]['lon'], det_storms[tt][0]['lon'][next_storm])
                storms[ed]['lat'] = np.append(storms[ed]['lat'], det_storms[tt][0]['lat'][next_storm])
                storms[ed]['amp'] = np.append(storms[ed]['amp'], det_storms[tt][0]['amp'][next_storm])
                storms[ed]['year'] = np.append(storms[ed]['year'], year[tt])
                storms[ed]['month'] = np.append(storms[ed]['month'], month[tt])
                storms[ed]['day'] = np.append(storms[ed]['day'], day[tt])
                storms[ed]['hour'] = np.append(storms[ed]['hour'], hour[tt])

                # Remove detected storm from list of storms available for assigment to existing trajectories

                unassigned.remove(next_storm)

                # Terminate storm otherwise

            else:

                storms[ed]['terminated'] = True

    # Create "new storms" from list of storms not assigned to existing trajectories

    if len(unassigned) > 0:

        for un in unassigned:

            storm_tmp = {}
            storm_tmp['lon'] = np.array([det_storms[tt][0]['lon'][un]])
            storm_tmp['lat'] = np.array([det_storms[tt][0]['lat'][un]])
            storm_tmp['amp'] = np.array([det_storms[tt][0]['amp'][un]])
            storm_tmp['type'] = det_storms[tt][0]['type'][un]
            storm_tmp['year'] = year[tt]
            storm_tmp['month'] = month[tt]
            storm_tmp['day'] = day[tt]
            storm_tmp['hour'] = hour[tt]
            storm_tmp['exist_at_start'] = False
            storm_tmp['terminated'] = False
            storms.append(storm_tmp)

    return storms


def strip_storms(tracked_storms, dt, d_tot_min=1000., d_ratio=0.6, dur_min=72):
    '''
    Following Klotzbach et al. (MWR, 2016) strip out storms with:
     1. A duration of less than dur_min (in hours). dt provides the
        time step of the track data (in hours).
     2. A total track length <= d_tot_min (short tracks)
     3. A start-to-end straight-line distance that is less than d_ratio
        times the total track length (meandering tracks).

    Use d_tot_min = 0, d_ratio = 0 and/or dur_min = 0 to avoid stripping out
    storms due to these criteria. It is recommended to use dur_min >= 6 or 12
    hours in order to remove a significant number of "storms" that appear due
    to high-frequency synoptic variability in the data.
    '''

    stripped_storms = []

    for ed in range(len(tracked_storms)):

        # 1. Remove storms which last less than dur_min hours
        if len(tracked_storms[ed]['lon']) <= dur_min/dt:
            continue

        # 2. Calculate total track length
        d_tot = 0
        for k in range(len(tracked_storms[ed]['lon'])-1):
            d_tot += latlon2km(tracked_storms[ed]['lon'][k], tracked_storms[ed]['lat'][k], tracked_storms[ed]['lon'][k+1], tracked_storms[ed]['lat'][k+1])

        # 3. Calcualate start-to-end straight-line track distance
        d_str = latlon2km(tracked_storms[ed]['lon'][0], tracked_storms[ed]['lat'][0], tracked_storms[ed]['lon'][-1], tracked_storms[ed]['lat'][-1])

        # Keep storms that satisfy the conditions 2 & 3
        if (d_tot >= d_tot_min) * ((d_str / d_tot) >= d_ratio):
            stripped_storms.append(tracked_storms[ed])

    return stripped_storms

def classify_storms(tracked_storms):
    '''
    Robert: This function classifies storms based on the condition that they form inside a bounding area
     and terminate outside of the formation area. At this moment we have supporting literature for
     Alberta Clippers and Colorado Lows. Remaining cyclones are classified as "other"
    '''
    clipper_zone = [[-115,50],
                    [-105,50],
                    [-105,55],
                    [-110,55],
                    [-110,60],
                    [-125,60],
                    [-125,55],
                    [-115,55]]
    northwst_zone = [[-125, 60],
                    [-115,60],
                    [-115,65],
                    [-125,65]]
    colorado_zone = [[-105, 40],
                    [-100,40],
                    [-100,35],
                    [-105,35]]
    basin_zone =    [[-120, 45],
                    [-115,45],
                    [-115,35],
                    [-120,35]]
    gom_zone =      [[-100, 25],
                    [-90,25],
                    [-90,30],
                    [-100,30]]
    eastcst_zone =  [[-80, 30],
                    [-75,30],
                    [-75,35],
                    [-65,35],
                    [-65,45],
                    [-70,45],
                    [-70,40],
                    [-80,40]]

    clipperPoly = mplPath.Path(clipper_zone)
    northwestPoly = mplPath.Path(northwst_zone)
    coloradoPoly = mplPath.Path(colorado_zone)
    basinPoly = mplPath.Path(basin_zone)
    gomPoly = mplPath.Path(gom_zone)
    eastCoastPoly = mplPath.Path(eastcst_zone)

    classified_storms = []

    for ed in range(len(tracked_storms)):
        start_lon = tracked_storms[ed]['lon'][0]
        start_lat = tracked_storms[ed]['lat'][0]
        end_lon = tracked_storms[ed]['lon'][-1]
        end_lat = tracked_storms[ed]['lat'][-1]

        startPoint = (start_lon, start_lat)
        endPoint = (end_lon, end_lat)
        if clipperPoly.contains_point(startPoint, radius=1e-9) and not clipperPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "Clipper"
        elif northwestPoly.contains_point(startPoint, radius=1e-9) and not northwestPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "Northwest"
        elif coloradoPoly.contains_point(startPoint, radius=1e-9) and not coloradoPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "Colorado"
        elif basinPoly.contains_point(startPoint, radius=1e-9) and not basinPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "GreatBasin"
        elif gomPoly.contains_point(startPoint, radius=1e-9) and not gomPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "GulfOfMexico"
        elif eastCoastPoly.contains_point(startPoint, radius=1e-9) and not eastCoastPoly.contains_point(endPoint, radius=1e-9):
            tracked_storms[ed]['classification'] = "EastCoast"
        else:
            tracked_storms[ed]['classification'] = "Other"

        classified_storms.append(tracked_storms[ed])

    return classified_storms

# Robert: Added function to calculate Bergeron value for cyclones
def calculate_bergeron(stormObject):
    bergeronList = []
    for i in range(len(stormObject['amp'])):
        pressures = stormObject['amp'][i:i+8]
        latitudes = stormObject['lat'][i:i+8]

        angFactor = np.sin(np.radians(60)) / np.sin(np.radians(latitudes))

        diffs = np.diff(pressures) / 100 #Convert Pa to mb
        negatives = np.sum(val for val in diffs if val < 0)	

        bFactor = (-1 * np.sum(negatives)) / (24)

        final = bFactor * angFactor

        bergeronList.append(bFactor)
    return bergeronList

def timevector(date_start, date_end):
    '''
    Generated daily time vector, along with year, month, day, day-of-year,
    and full date information, given start and and date. Format is a 3-element
    list so that a start date of 3 May 2005 is specified date_start = [2005,5,3]
    Note that day-of year (doy) is [0 to 59, 61 to 366] for non-leap years and [0 to 366]
    for leap years.
    returns: t, dates, T, year, month, day, doy
    '''
    # Time vector
    t = np.arange(date(date_start[0],date_start[1],date_start[2]).toordinal(),date(date_end[0],date_end[1],date_end[2]).toordinal()+1)
    T = len(t)
    # Date list
    dates = [date.fromordinal(tt.astype(int)) for tt in t]
    # Vectors for year, month, day-of-month
    year = np.zeros((T))
    month = np.zeros((T))
    day = np.zeros((T))
    for tt in range(T):
        year[tt] = date.fromordinal(t[tt]).year
        month[tt] = date.fromordinal(t[tt]).month
        day[tt] = date.fromordinal(t[tt]).day
    year = year.astype(int)
    month = month.astype(int)
    day = day.astype(int)
    # Leap-year baseline for defining day-of-year values
    year_leapYear = 2012 # This year was a leap-year and therefore doy in range of 1 to 366
    t_leapYear = np.arange(date(year_leapYear, 1, 1).toordinal(),date(year_leapYear, 12, 31).toordinal()+1)
    dates_leapYear = [date.fromordinal(tt.astype(int)) for tt in t_leapYear]
    month_leapYear = np.zeros((len(t_leapYear)))
    day_leapYear = np.zeros((len(t_leapYear)))
    doy_leapYear = np.zeros((len(t_leapYear)))
    for tt in range(len(t_leapYear)):
        month_leapYear[tt] = date.fromordinal(t_leapYear[tt]).month
        day_leapYear[tt] = date.fromordinal(t_leapYear[tt]).day
        doy_leapYear[tt] = t_leapYear[tt] - date(date.fromordinal(t_leapYear[tt]).year,1,1).toordinal() + 1
    # Calculate day-of-year values
    doy = np.zeros((T))
    for tt in range(T):
        doy[tt] = doy_leapYear[(month_leapYear == month[tt]) * (day_leapYear == day[tt])]
    doy = doy.astype(int)

    return t, dates, T, year, month, day, doy

## Storm Detection
The first step of the analysis is to run the storm detection block. This code loops through the incoming data and runs the eddy detection code on it, a giant list object containing each time step and properties of each eddy at each time step is written out.

In [ ]:
'''
This is a multiprocessing-ready function that is used to delegate tasks
across the multiprocessing.map function to allow for quicker processing.
'''
def run_detection(slp, idx, size, lon, lat):
    processStart = time.time()
    lon_storms_a = []
    lat_storms_a = []
    amp_storms_a = []
    lon_storms_c = []
    lat_storms_c = []
    amp_storms_c = []
    # anti-cyclones
    lon_storms, lat_storms, amp = detect_storms(slp, lon, lat, res=2, Npix_min=9, cyc='anticyclonic')
    lon_storms_a.append(lon_storms)
    lat_storms_a.append(lat_storms)
    amp_storms_a.append(amp)
    # cyclones
    lon_storms, lat_storms, amp = detect_storms(slp, lon, lat, res=2, Npix_min=9, cyc='cyclonic')
    lon_storms_c.append(lon_storms)
    lat_storms_c.append(lat_storms)
    amp_storms_c.append(amp)
    # Write out
    storms = storms_list(lon_storms_a, lat_storms_a, amp_storms_a, lon_storms_c, lat_storms_c, amp_storms_c)
    processEnd = time.time()
    print("Time step completed (" + str(idx) + " / " + str(size) + "), Elapsed Time: " + time.strftime("%H:%M:%S", time.gmtime(processEnd - processStart)))
    return {idx: storms}

This next block of code runs the detection routine. 

**NOTE:** This will take a lot of time depending on how many files you are trying to process, how big the files are, and how many processors you have on your system

In [ ]:
#
# Load in slp data and lat/lon coordinates
#
print("Program Start...")

# Parameters
## NOTE: MAKE SURE YOU EDIT THIS LINE!!!!
## THIS IS WHERE THE PROGRAM WILL LOOK FOR DATA
dataDir = ''
# This is the variable that is pulled from the netCDF file
var = 'mslet'

# Generate date and hour vectors
yearStart = 1979
yearEnd = 2020 

# Load lat, lon
filename = dataDir + "mslet." + str(yearStart) + ".nc"
print("Loading in first netCDF file to populate arrays... " + str(filename[dataset]))
fileobj = Dataset(filename[dataset], 'r')
lon = fileobj.variables['lon'][:].astype(float)
lat = fileobj.variables['lat'][:].astype(float)
fileobj.close()

bigListStorms = []

print("Spawning a multiprocessing pool, " + str(os.cpu_count()) + " processors detected.")
pool = multiprocessing.Pool(os.cpu_count())

print("Entering loop, beginning timer.")
fullStart = time.time()
yListTime = []

# Create empty arrays to hold the data
year = np.zeros((0,))
month = np.zeros((0,))
day = np.zeros((0,))
hour = np.zeros((0,))

for yr in range(yearStart, yearEnd+1):
    save_storms = {}
    current_list_storms = []

    innerTimeStart = time.time()

    filename = dataDir + "mslet." + str(yr) + ".nc"
    fileobj = Dataset(filename[dataset], 'r')
    timeAr = fileobj.variables['time'][:]
    time_ordinalDays = timeAr/24. + date(1800,1,1).toordinal()
    year = np.append(year, [date.fromordinal(np.floor(time_ordinalDays[tt]).astype(int)).year for tt in range(len(timeAr))])
    month = np.append(month, [date.fromordinal(np.floor(time_ordinalDays[tt]).astype(int)).month for tt in range(len(timeAr))])
    day = np.append(day, [date.fromordinal(np.floor(time_ordinalDays[tt]).astype(int)).day for tt in range(len(timeAr))])
    hour = np.append(hour, (np.mod(time_ordinalDays, 1)*24).astype(int))
    slp0 = fileobj.variables[var[dataset]][:].astype(float)
    fileobj.close()

    # Run the multiprocessed detection function, merge the returned dictionary object with our local one.
    out_dict = pool.starmap(run_detection, zip(slp0, np.arange(slp0.shape[0]), itertools.repeat(slp0.shape[0]), itertools.repeat(lon), itertools.repeat(lat)))
    ro_inner = time.time()
    for iDict in out_dict: #Note: starmap returns as a 0-length list ie [return] with the Nth element being each input
        save_storms = {**save_storms, **iDict} 
    # Order the final list correctly
    for i in range(slp0.shape[0]):
        current_list_storms.append(save_storms[i])
    ro_outer = time.time() - ro_inner
    print(str(yr) + " List Reordering Completed, Elapsed Time: " + time.strftime("%H:%M:%S", time.gmtime(ro_outer)))
    # Append to the big list, then save.
    bigListStorms.append(current_list_storms)
    innerTimeEnd = time.time()
    yListTime.append((innerTimeEnd - innerTimeStart))
    print(str(yr) + " Completed, Elapsed Time: " + time.strftime("%H:%M:%S", time.gmtime(innerTimeEnd - innerTimeStart)))

    np.savez('storm_det_slp', storms=bigListStorms, year=year, month=month, day=day, hour=hour)

pool.close() 
pool.join()

fullEnd = time.time()
print("Program Completed, Elapsed Time: " + time.strftime("%H:%M:%S", time.gmtime(fullEnd - fullStart)))

for i, yi in enumerate(yListTime):
    print("Processing Time (" + str(yearStart + i) + "): " + time.strftime("%H:%M:%S", time.gmtime(yi)))

## Storm Tracking
After completing the above storm detection code, you will have a *storm_det_slp.npz* file generated, as mentioned this contains a giant list of time steps from which all eddies are marked within.

This code block runs the eddy tracking routine, and applies the tracking restrictions as defined by Klotzbach et al. (MWR, 2016) to remove any eddies that do not behave like storms. Additional criterion may be considered and can be added in at the specified point.

No multiprocessing was introduced for this routine, so it will take some time to run through completely, although for most cases, I noted that ~40 years of data completes in about 2 hours.

In [ ]:
print("Program Start...")

# Pass the storm_det_slp file generated in the previous block to this
# call here.
data = np.load('storm_det_slp.npz', encoding='latin1', allow_pickle=True)

det_storms = data['storms']
year = data['year']
month = data['month']
day = data['day']
hour = data['hour']

# Initialize storms discovered at first time step

storms = storms_init(det_storms, year, month, day, hour)

# Stitch storm tracks together at future time steps
T = len(det_storms) # number of time steps
print("Preparing to enter loop, there are " + str(T) + " total time steps to evaluate")
processStart = time.time()
avgLoop = 0
for tt in range(0, T):
    # Track storms from time step tt-1 to tt and update corresponding tracks and/or create new storms
    loopTimeStart = time.time()
    storms = track_storms(storms, det_storms, tt, year, month, day, hour, dt=3)
    loopTimeEnd = time.time()
    procTime = loopTimeEnd - loopTimeStart
    avgLoop += procTime

    if(tt % 1000 == 0 and tt != 0):
        print("Evaluation: " + str(tt) + "/" + str(T) + " - Avg. Time: " + 
        time.strftime("%H:%M:%S", time.gmtime(avgLoop / tt))
        + ", Est. Completion: " + time.strftime("%H:%M:%S", time.gmtime(avgLoop * (T - tt))))
processEnd = time.time()
print("Loop completed, total time: " + time.strftime("%H:%M:%S", time.gmtime(processEnd - processStart)))

# Add keys for storm age and flag if storm was still in existence at end of run
for ed in range(len(storms)):
    storms[ed]['age'] = len(storms[ed]['lon'])

# Robert: Added Classification here
print("Classifying Storms...")
storms = classify_storms(storms)

# Strip storms based on track lengths (dt = 3hr, d_tot_min = 500km, dur_min = 24hr)
print("Removing Storms that do not meet criteron...")
storms = strip_storms(storms, dt=3, d_tot_min=500., d_ratio=0.6, dur_min=24)

# Save tracked storm data
print("Saving output file...")
np.savez('storm_track_slp', storms=storms)

print("Program Complete.")